In [ ]:
#Define SoH: state-of-health, calculated by Qmax/Ct, where Qmax = maximum charge of battery, and Ct = rated capacity

#Cc / Ci (science direct: Zhang Fan, et al), where Cc = is maximum available capacity of current cycle, and Ci is maximum capacity of init

#Rated battery capacity for 2.7V = 1.856487 Ah

In [18]:
import pandas as pd

NASA_clean = pd.read_csv('AllData_FixedSub.csv')
NASA_clean.head()

current_hr = NASA_clean.loc[NASA_clean['cycle_type']=='discharge']

current_hr = current_hr.loc[current_hr['Voltage_measured'] != 2.7]

current_hr.head()

#Currently a WIP, but this isolates important variables for SOH in preparation for manual calculation.

#for the model: 

#start working with RUL by predicting the total runtime of battery, given different conditions (take all data: #1, last - 1)
#then work with SOH by predicting a condition %, given all initial info

,Voltage_measured,Current_measured,Temperature_measured,Current_charge,Voltage_charge,Time,cycle_type,start_time,ambient_temperature,Current_load,...,Rct,Corrected_time,current_impedance,ΔT,Δt,temp_gradient,Heat_generated,heat_flux,heat_loss_rate,convective_heat_transfer
789,4.191492,-0.004902,24.330034,NaN,NaN,0.000,discharge,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,24,-0.0006,...,NaN,7597.875,0.2,-0.177007,0.000,-inf,0.000000,inf,inf,0.003093
790,4.190749,-0.001478,24.325993,NaN,NaN,16.781,discharge,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,24,-0.0006,...,NaN,7614.656,0.2,-0.004040,16.781,-0.000241,0.000007,0.000104,4.348414e-07,0.003046
791,3.974871,-2.012528,24.389085,NaN,NaN,35.703,discharge,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,24,-1.9982,...,NaN,7633.578,0.2,0.063092,18.922,0.003334,15.327843,-0.001434,-6.021753e-06,0.003799
792,3.951717,-2.013979,24.544752,NaN,NaN,53.781,discharge,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,24,-1.9982,...,NaN,7651.656,0.2,0.155667,18.078,0.008611,14.665282,-0.003703,-1.555122e-05,0.005786
793,3.934352,-2.011144,24.731385,NaN,NaN,71.922,discharge,[2.0080e+03 4.0000e+00 2.0000e+00 1.5000e+01 2...,24,-1.9982,...,NaN,7669.797,0.2,0.186633,18.141,0.010288,14.674975,-0.004424,-1.857996e-05,0.008362


WIP prediction algorithm for remaining useful life, in seconds. Inputs will be every useful variable, but output will be configured to release more useful information about remaining life.

In [ ]:
#Start out with a simple LinearRegression model (base). Import various regression packages: 

my_test = pd.read_csv('AllData_FixedSub.csv')

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Y = my_test["Corrected_time"]
X = my_test.drop(["Current_charge","Voltage_charge","Time","cycle_type","start_time","Sense_current","Battery_current",
"Current_ratio","Battery_impedance","Rectified_Impedance", "Re", "Rct"], axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25)

regressor = LinearRegression()
regressor.fit(X_train, Y_train)

Y_pred_train = regressor.predict(X_train)

Y_pred_test = regressor.predict(X_test)

#Develop upon error statistics for further analysis.

train_review = pd.DataFrame({"Model Prediction": Y_pred_train,
             "Expected/Real values": Y_train})

test_review = pd.DataFrame({"Model Prediction": Y_pred_test,
             "Expected/Real values": Y_test, "Absolute Error": (abs(Y_pred_test - Y_test))})

my_error = mean_squared_error(Y_test, Y_pred_test, squared=False)

r2 = r2_score(Y_test, Y_pred_test)
print(my_error, r2)

test_review.info